# **Target Mean Encoding**

In [10]:
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore')

In [14]:
from sklearn.datasets import fetch_openml
data = fetch_openml(name='mushroom')
print(data.url)
data.keys()

https://www.openml.org/d/24


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [16]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


In [17]:
df.target.value_counts()

target
e    4208
p    3916
Name: count, dtype: int64

In [19]:
fe = df.groupby('cap-surface').size()/len(df)
fe

cap-surface
f    0.285574
g    0.000492
s    0.314623
y    0.399311
dtype: float64

In [21]:
df['cap-surface'].map(fe)

0       0.314623
1       0.314623
2       0.314623
3       0.399311
4       0.314623
          ...   
8119    0.314623
8120    0.314623
8121    0.314623
8122    0.399311
8123    0.314623
Name: cap-surface, Length: 8124, dtype: category
Categories (4, float64): [0.285574, 0.000492, 0.314623, 0.399311]

In [25]:
from sklearn import preprocessing

# Create an instance of LabelEncoder
la = preprocessing.LabelEncoder()

# Assuming 'df' is your DataFrame and 'target' is the column with categorical labels
# Now use the instance 'la' to call fit_transform
df['target'] = la.fit_transform(df['target'])
df

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,1
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,0
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,0
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,1
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,0
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,0
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,0
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,1


### **1-1. Transform single value into target mean encoding**

In [26]:
df.groupby (['cap-surface'])['target'].mean()

cap-surface
f    0.327586
g    1.000000
s    0.552426
y    0.536375
Name: target, dtype: float64

In [27]:
target_mean = df.groupby (['cap-surface'])['target'].mean()
df['cap-surface'].map(target_mean) # map 유용한 기능

0       0.552426
1       0.552426
2       0.552426
3       0.536375
4       0.552426
          ...   
8119    0.552426
8120    0.552426
8121    0.552426
8122    0.536375
8123    0.552426
Name: cap-surface, Length: 8124, dtype: category
Categories (4, float64): [0.327586, 1.000000, 0.552426, 0.536375]

### **1-2. Transform multiple values into target mean encoding**

In [33]:
tm = pd.DataFrame(df.groupby(['cap-surface', 'cap-color'])['target'].mean()).reset_index()
tm

,cap-surface,cap-color,target
0,f,b,NaN
1,f,c,0.000000
2,f,e,0.000000
3,f,g,0.399103
4,f,n,0.016529
5,f,p,1.000000
6,f,r,NaN
7,f,u,NaN
8,f,w,0.125000
9,f,y,0.932584


In [31]:
df.merge(tm, on =['cap-surface', 'cap-color'], how='left', suffixes=('', '_tm'))

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target,target_tm
0,x,s,n,t,p,f,c,n,k,e,...,w,p,w,o,p,k,s,u,1,0.582160
1,x,s,y,t,a,f,c,b,k,e,...,w,p,w,o,p,n,n,g,0,0.000000
2,b,s,w,t,l,f,c,b,n,e,...,w,p,w,o,p,n,n,m,0,0.366906
3,x,y,w,t,p,f,c,n,n,e,...,w,p,w,o,p,k,s,u,1,0.357143
4,x,s,g,f,n,f,w,b,k,t,...,w,p,w,o,e,n,a,g,0,0.385542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,p,o,o,p,b,c,l,0,0.582160
8120,x,s,n,f,n,a,c,b,y,e,...,o,p,n,o,p,b,v,l,0,0.582160
8121,f,s,n,f,n,a,c,b,n,e,...,o,p,o,o,p,b,c,l,0,0.582160
8122,k,y,n,f,y,f,c,n,b,t,...,w,p,w,o,e,w,v,l,1,0.544304


## **[Target Encoder](https://contrib.scikit-learn.org/category_encoders/targetencoder.html)**

In [3]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.0 MB/s eta 0:00:00


In [34]:
df.head()

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,1
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,0
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,0
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,1
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,0


### **1-3. Transform single value into target mean encoding with target encoding module**

In [35]:
target_mean = df.groupby (['cap-surface'])['target'].mean()
df['cap-surface'].map(target_mean) # map 유용한 기능

0       0.552426
1       0.552426
2       0.552426
3       0.536375
4       0.552426
          ...   
8119    0.552426
8120    0.552426
8121    0.552426
8122    0.536375
8123    0.552426
Name: cap-surface, Length: 8124, dtype: category
Categories (4, float64): [0.327586, 1.000000, 0.552426, 0.536375]

In [36]:
# Initialize the encoder
encoder = TargetEncoder()
# Fit and transform the categories
encoder.fit_transform(df['cap-surface'], df['target'])
# Display the encoded dataframe
# df['Category_Encoded'] = encoder.fit_transform(df['cap-surface'], df['target'])
# df

,cap-surface
0,0.552426
1,0.552426
2,0.552426
3,0.536375
4,0.552426
...,...
8119,0.552426
8120,0.552426
8121,0.552426
8122,0.536375


### **1-4. Transform multiple values into target mean encoding with target encoding module**

In [42]:
df['interaction'] = df['cap-surface'].astype(str) + '_' + df['cap-color'].astype(str)
df['interaction']

0       s_n
1       s_y
2       s_w
3       y_w
4       s_g
       ... 
8119    s_n
8120    s_n
8121    s_n
8122    y_n
8123    s_n
Name: interaction, Length: 8124, dtype: object

In [44]:
# Fit and transform the interaction feature
encoder.fit_transform(df['interaction'], df['target']).round(2)

,interaction
0,0.58
1,0.00
2,0.37
3,0.36
4,0.39
...,...
8119,0.58
8120,0.58
8121,0.58
8122,0.54


## **Appendix**

In [1]:
('col %d' % i for i in range(20))

<generator object <genexpr> at 0x7eb78733f6f0>

In [ ]:
myt = ()
for i in range(20):
    print(f"col {i}")

col 0
col 1
col 2
col 3
col 4
col 5
col 6
col 7
col 8
col 9
col 10
col 11
col 12
col 13
col 14
col 15
col 16
col 17
col 18
col 19


In [ ]:
myt = []
for i in range(20):
    print(f"col {i}")
    myt += [f"col {i}",]

col 0
col 1
col 2
col 3
col 4
col 5
col 6
col 7
col 8
col 9
col 10
col 11
col 12
col 13
col 14
col 15
col 16
col 17
col 18
col 19


In [ ]:
myt

['col 0',
 'col 1',
 'col 2',
 'col 3',
 'col 4',
 'col 5',
 'col 6',
 'col 7',
 'col 8',
 'col 9',
 'col 10',
 'col 11',
 'col 12',
 'col 13',
 'col 14',
 'col 15',
 'col 16',
 'col 17',
 'col 18',
 'col 19']